## Text splitters & RAG


Building effective RAG systems starts with splitting documents into meaningful, searchable chunks.

LangChain’s text splitters help break large PDFs, reports, and manuals into clean segments that improve embedding quality and reduce hallucinations.

This tutorial introduces the essentials of text splitting, embedding, and retrieval—giving you the foundations to build accurate, efficient RAG pipelines.

In [1]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    SentenceTransformersTokenTextSplitter,
    TextSplitter,
    TokenTextSplitter,
)


In [2]:

# Load environment variables from .env
load_dotenv()

True

In [3]:
# Define the directory containing the text file and the persistent directory
current_dir = os.path.dirname(os.path.abspath("__file__"))
file_path = os.path.join(current_dir, "data", "IOT.pdf")
db_dir = os.path.join(current_dir, "db")

#### load pdf

In [5]:
# load pdf
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader(file_path)
documents = loader.load()



* Reference link for built-in document loader in langchain
* https://docs.langchain.com/oss/python/integrations/document_loaders

#### Text splitters

In [6]:
# split document content

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [7]:
# Display information about the split documents
print("\n--- Document Chunks Information ---")
print(f"Number of document chunks: {len(docs)}")
print(f"Sample chunk:\n{docs[0].page_content}\n")


--- Document Chunks Information ---
Number of document chunks: 4
Sample chunk:
Why is Internet of Things (IoT) so important? 
Over the past few years, IoT has become one of the most important technologies of the 
21st century. Now that we can connect everyday objects—kitchen appliances, cars, 
thermostats, baby monitors—to the internet via embedded devices, seamless 
communication is possible between people, processes, and things. 
 
By means of low-cost computing, the cloud, big data, analytics, and mobile 
technologies, physical things can share and collect data with minimal human 
intervention. In this hyperconnected world, digital systems can record, monitor, and 
adjust each interaction between connected things. The physical world meets the digital 
world—and they cooperate. 
 
What technologies have made IoT possible? 
While the idea of IoT has been in existence for a long time, a collection of recent 
advances in a number of different technologies has made it practical.



In [8]:
# Create embeddings
print("\n--- Creating embeddings ---")
embeddings = OpenAIEmbeddings()
print("\n--- Finished creating embeddings ---")


--- Creating embeddings ---

--- Finished creating embeddings ---


In [9]:
# Create the vector store and persist it automatically

# Function to create and persist vector store
def create_vector_store(docs, store_name):
    persistent_directory = os.path.join(db_dir, store_name)
    if not os.path.exists(persistent_directory):
        print(f"\n--- Creating vector store {store_name} ---")
        db = Chroma.from_documents(
            docs, embeddings, persist_directory=persistent_directory
        )
        print(f"--- Finished creating vector store {store_name} ---")
    else:
        print(
            f"Vector store {store_name} already exists. No need to initialize.")



#### 1. Character-based Splitting
* Splits text into chunks based on a specified number of characters.
* Useful for consistent chunk sizes regardless of content structure

In [10]:

print("\n--- Using Character-based Splitting ---")
char_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
char_docs = char_splitter.split_documents(documents)
create_vector_store(char_docs, "chroma_db_char")


--- Using Character-based Splitting ---

--- Creating vector store chroma_db_char ---
--- Finished creating vector store chroma_db_char ---


#### 2. Sentence-based Splitting
* Splits text into chunks based on sentences, ensuring chunks end at sentence boundaries.

* Counts tokens using the SentenceTransformer model’s own tokenizer so chunks never exceed the model’s maximum token limit.

* Tries to split text first at meaningful breakpoints—paragraphs (\n\n), sentences (., ?, !)—similar to a recursive splitting strategy.

* If a paragraph or sentence is too long, it falls back to finer boundaries (single \n, then spaces, then characters) until the chunk fits the token limit.

* Uses the optional chunk_overlap parameter to repeat a portion of tokens across chunks to maintain context continuity.

* Ideal for maintaining semantic coherence within chunks.

In [11]:

print("\n--- Using Sentence-based Splitting ---")
sent_splitter = SentenceTransformersTokenTextSplitter(chunk_size=1000)
sent_docs = sent_splitter.split_documents(documents)
create_vector_store(sent_docs, "chroma_db_sent")



--- Using Sentence-based Splitting ---


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\vyanktesh.l\Documents\FDE\genai_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vyanktesh.l\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


--- Creating vector store chroma_db_sent ---
--- Finished creating vector store chroma_db_sent ---


#### 3. Token-based Splitting
* Splits text into chunks based on tokens (words or subwords), using tokenizers
* Useful for transformer models with strict token limits.

In [12]:

print("\n--- Using Token-based Splitting ---")
token_splitter = TokenTextSplitter(chunk_overlap=0, chunk_size=512)
token_docs = token_splitter.split_documents(documents)
create_vector_store(token_docs, "chroma_db_token")



--- Using Token-based Splitting ---

--- Creating vector store chroma_db_token ---
--- Finished creating vector store chroma_db_token ---


#### 4. Recursive Character-based Splitting
* Attempts to split text at natural boundaries (sentences, paragraphs) within character limit.
* Balances between maintaining coherence and adhering to character limits.

In [13]:

print("\n--- Using Recursive Character-based Splitting ---")
rec_char_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100)
rec_char_docs = rec_char_splitter.split_documents(documents)
create_vector_store(rec_char_docs, "chroma_db_rec_char")



--- Using Recursive Character-based Splitting ---

--- Creating vector store chroma_db_rec_char ---
--- Finished creating vector store chroma_db_rec_char ---


#### 5. Custom Splitting
* Allows creating custom splitting logic based on specific requirements.
* Useful for documents with unique structure that standard splitters can't handle.

In [14]:

print("\n--- Using Custom Splitting ---")


class CustomTextSplitter(TextSplitter):
    def split_text(self, text):
        # Custom logic for splitting text
        return text.split("\n\n")  # Example: split by paragraphs


custom_splitter = CustomTextSplitter()
custom_docs = custom_splitter.split_documents(documents)
create_vector_store(custom_docs, "chroma_db_custom")



--- Using Custom Splitting ---

--- Creating vector store chroma_db_custom ---
--- Finished creating vector store chroma_db_custom ---


In [15]:
# Function to query a vector store
def query_vector_store(store_name, query):
    persistent_directory = os.path.join(db_dir, store_name)
    if os.path.exists(persistent_directory):
        print(f"\n--- Querying the Vector Store {store_name} ---")
        db = Chroma(
            persist_directory=persistent_directory, embedding_function=embeddings
        )
        retriever = db.as_retriever(
            search_type="similarity_score_threshold",
            search_kwargs={"k": 1, "score_threshold": 0.1},
        )
        relevant_docs = retriever.invoke(query)
        # Display the relevant results with metadata
        print(f"\n--- Relevant Documents for {store_name} ---")
        for i, doc in enumerate(relevant_docs, 1):
            print(f"Document {i}:\n{doc.page_content}\n")
            if doc.metadata:
                print(f"Source: {doc.metadata.get('source', 'Unknown')}\n")
    else:
        print(f"Vector store {store_name} does not exist.")


In [16]:

# Define the user's question
query = "What technologies have made IoT possible?"


In [17]:
# Query each vector store
query_vector_store("chroma_db_char", query)
query_vector_store("chroma_db_sent", query)
query_vector_store("chroma_db_token", query)
query_vector_store("chroma_db_rec_char", query)
query_vector_store("chroma_db_custom", query)



--- Querying the Vector Store chroma_db_char ---


C:\Users\vyanktesh.l\AppData\Local\Temp\ipykernel_43180\760185032.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  db = Chroma(



--- Relevant Documents for chroma_db_char ---
Document 1:
Why is Internet of Things (IoT) so important? 
Over the past few years, IoT has become one of the most important technologies of the 
21st century. Now that we can connect everyday objects—kitchen appliances, cars, 
thermostats, baby monitors—to the internet via embedded devices, seamless 
communication is possible between people, processes, and things. 
 
By means of low-cost computing, the cloud, big data, analytics, and mobile 
technologies, physical things can share and collect data with minimal human 
intervention. In this hyperconnected world, digital systems can record, monitor, and 
adjust each interaction between connected things. The physical world meets the digital 
world—and they cooperate. 
 
What technologies have made IoT possible? 
While the idea of IoT has been in existence for a long time, a collection of recent 
advances in a number of different technologies has made it practical. 
 
Access to low-cost, low-p

#### 🧩 Document Structure–Based Splitting
Some documents have an inherent structure — such as HTML, Markdown, or JSON files.
Splitting based on their structure helps preserve meaning and improve downstream performance.

✅ Key Benefits
* Preserves the logical organization of the document
* Maintains context within each chunk
* Improves retrieval and summarization accuracy

🧱 Examples
* Markdown → Split by headers (#, ##, ###)
* HTML → Split by tags (&lt; div&gt;, &lt;p&gt;, &lt;section&gt;)
* JSON → Split by object or array elements
* Code → Split by functions, classes, or logical blocks


⚙️ Available Splitters
* SplitMarkdown
* SplitJSON
* SplitCode
* SplitHTML

In [18]:
from langchain_text_splitters import HTMLSemanticPreservingSplitter

html_string = """
<!DOCTYPE html>
<html>
    <body>
        <div>
            <h1>Section 1</h1>
            <p>This section contains an important table and list that should not be split across chunks.</p>
            <table>
                <tr>
                    <th>Item</th>
                    <th>Quantity</th>
                    <th>Price</th>
                </tr>
                <tr>
                    <td>Apples</td>
                    <td>10</td>
                    <td>$1.00</td>
                </tr>
                <tr>
                    <td>Oranges</td>
                    <td>5</td>
                    <td>$0.50</td>
                </tr>
                <tr>
                    <td>Bananas</td>
                    <td>50</td>
                    <td>$1.50</td>
                </tr>
            </table>
            <h2>Subsection 1.1</h2>
            <p>Additional text in subsection 1.1 that is separated from the table and list.</p>
            <p>Here is a detailed list:</p>
            <ul>
                <li>Item 1: Description of item 1, which is quite detailed and important.</li>
                <li>Item 2: Description of item 2, which also contains significant information.</li>
                <li>Item 3: Description of item 3, another item that we don't want to split across chunks.</li>
            </ul>
        </div>
    </body>
</html>
"""

headers_to_split_on = [("h1", "Header 1"), ("h2", "Header 2")]

splitter = HTMLSemanticPreservingSplitter(
    headers_to_split_on=headers_to_split_on,
    max_chunk_size=50,
    elements_to_preserve=["table", "ul"],
)

documents = splitter.split_text(html_string)
print(documents)

[Document(metadata={'Header 1': 'Section 1'}, page_content='This section contains an important table and list'), Document(metadata={'Header 1': 'Section 1'}, page_content='that should not be split across chunks.'), Document(metadata={'Header 1': 'Section 1'}, page_content='Item Quantity Price Apples 10 $1.00 Oranges 5 $0.50 Bananas 50 $1.50'), Document(metadata={'Header 2': 'Subsection 1.1'}, page_content='Additional text in subsection 1.1 that is'), Document(metadata={'Header 2': 'Subsection 1.1'}, page_content='separated from the table and list. Here is a'), Document(metadata={'Header 2': 'Subsection 1.1'}, page_content="detailed list: Item 1: Description of item 1, which is quite detailed and important. Item 2: Description of item 2, which also contains significant information. Item 3: Description of item 3, another item that we don't want to split across chunks.")]


C:\Users\vyanktesh.l\AppData\Local\Temp\ipykernel_43180\603221681.py:47: LangChainBetaWarning: The class `HTMLSemanticPreservingSplitter` is in beta. It is actively being worked on, so the API may change.
  splitter = HTMLSemanticPreservingSplitter(


In [19]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

In [20]:
PYTHON_CODE = """
def hello_world():
    print("Hello, World!")

# Call the function
hello_world()
"""
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=50, chunk_overlap=0
)
python_docs = python_splitter.create_documents([PYTHON_CODE])
python_docs

[Document(metadata={}, page_content='def hello_world():\n    print("Hello, World!")'),
 Document(metadata={}, page_content='# Call the function\nhello_world()')]

#To view the full list of supported languages:


[e.value for e in Language]

### Retrieval and Generation:
The generation step involves combining the information retrieved from the indexed chunks with a language model to generate a final output.

We can put it all together into a chain that takes a question, retrieves relevant documents, constructs a prompt, passes that to a model, and parses the output.

In [21]:
store_name="chroma_db_char"
persistent_directory = os.path.join(db_dir, store_name)
db = Chroma(persist_directory=persistent_directory, embedding_function=embeddings)
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.1},
)

In [22]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini"
)



In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


parser = StrOutputParser()

This function runs a RAG pipeline by retrieving relevant documents, assembling their content into a context string, and passing it to the prompt. The prompt, LLM, and parser form a RAG chain that generates a final answer grounded in the retrieved context.

In [24]:

def rag_chain(question):
    docs = retriever.invoke(question)
    context = "\n\n".join([d.page_content for d in docs])
    
    chain = prompt | llm | parser
    return chain.invoke({"context": context, "input": question})

In [25]:
# Define the user's question
query = "What technologies have made IoT possible?"

In [26]:

rag_chain(query)

'The technologies that have made IoT possible include low-cost, low-power sensor technology for affordable and reliable device connectivity, various network protocols for efficient data transfer, and cloud computing platforms that provide scalable infrastructure. Advances in machine learning and analytics allow for faster insights from the vast data collected. Additionally, developments in conversational artificial intelligence contribute to user-friendly IoT devices.'